# GlobustVP Demo

This notebook demonstrates how to use **GlobustVP** to estimate vanishing points from a real image.

In [ ]:
%load_ext autoreload
%autoreload 2

import time
import numpy as np

from globustvp.solver.core import globustvp
from globustvp.utils.io import load_image_and_gray
from globustvp.utils.geometry import (
    normalize_lines,
    compute_backprojection_normals,
    compute_line_uncertainties
)
from globustvp.utils.plot import plot_lines_on_image, visualize_line_vp_associations
from globustvp.utils.line_processing import detect_and_format_lines

## 1. Load input image

In [ ]:
print("📷 Loading image...")
img, gray = load_image_and_gray("assets/indoor.jpg")

## 2. Detect and filter line segments using LSD and dominant orientation clustering

In [ ]:
print("🧪 Running LSD line detection and filtering...")
lines_2D = detect_and_format_lines(gray_image=gray, min_length=30)
print(f"✅ {lines_2D.shape[1]} lines remain after filtering.")

## 3. Visualize filtered line segments

In [ ]:
plot_lines_on_image(image=img, lines=lines_2D)

## 4. Normalize lines with camera intrinsics (adjust if known)

In [ ]:
print("📐 Normalizing lines using camera intrinsics...")
K = np.array([[674.917975164175, 0, 307.551305282635],
              [0, 674.917975164175, 251.454244960136],
              [0, 0, 1]])

normalized_lines = normalize_lines(K, lines_2D).T
print("✅ Normalization complete.")

## 5. Convert line segments to plane normals via back-projection

In [ ]:
print("🧮 Converting lines to back-projection normals...")
para_lines = compute_backprojection_normals(normalized_lines)
print("✅ Generated", para_lines.shape[0], "normals.")

## 6. Assign uncertainty for lines

In [ ]:
print("📏 Assigning uncertainty weights...")
uncertainty = compute_line_uncertainties(normalized_lines, K, use_uncertainty=True)

## 7. Run GlobustVP solver using SCS optimizer

In [ ]:
param = {
    "line_num": lines_2D.shape[1],
    "vanishing_point_num": 3,
    "c": 0.03,
    "sample_line_num": 4,
    "is_fast_solver": True,
    "eigen_threshold": 1,
    "solver": "SCS",
    "solver_opts": {"eps_abs": 1e-12, "eps_rel": 1e-12},
    "K": K
}

print("🚀 Running GlobustVP solver...")
t_start = time.time()
status, est_vps, est_corrs = globustvp(normalized_lines, para_lines, uncertainty, param)
t_end = time.time()

if status:
    print("✅ GlobustVP succeeded in {:.4f} seconds".format(t_end - t_start))
    print("🧭 Estimated vanishing points:\n", est_vps)
else:
    print("❌ GlobustVP failed to converge.")

## 8. Visualize line-VP association result

In [ ]:
print("🎨 Plotting vanishing point estimation result...")
visualize_line_vp_associations(image=img, lines=lines_2D, est_corrs=est_corrs)